In [0]:
import numpy as np

In [0]:
class Ban:
    def init_ban(self):
        first_board = np.array([[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]])
        first_board[4][4] = first_board[5][5] = 2
        first_board[4][5] = first_board[5][4] = 1
        return first_board

    def stone(self,ishi):
        if ishi == 0:
            return '・'
        elif ishi == 1:
            return '●'
        else:
            return'○'
    def draw_ban(self,board):
        print('  a  b  c  d  e  f  g  h ')
        print('  ------------------------')
        for i in range(1,9):
            print('%d|'%i, end='')
            for j in range(1,9):
                print(self.stone(board[i][j]),end=' ')
            print('|')
        print('  ------------------------')


In [0]:
class Play:
    def count_turn_over(self, board, color, y, x, d, e):
        i = 1
        while board[y + i * d][x + i * e] == (3 - color):
            i += 1
        if board[y + i * d][x + i * e] == color:
            return i - 1
        else:
            return 0
    def is_legal(self,board, color, y, x):
        if x < 1 or y > 8 or x < 1 or y > 8: return 0
        if board[y][x] != 0 :return 0
        if self.count_turn_over(board, color, y, x, -1, 0): return 1
        if self.count_turn_over(board, color, y, x,  1, 0): return 1
        if self.count_turn_over(board, color, y, x,  0,-1): return 1
        if self.count_turn_over(board, color, y, x,  0, 1): return 1
        if self.count_turn_over(board, color, y, x, -1,-1): return 1
        if self.count_turn_over(board, color, y, x, -1, 1): return 1
        if self.count_turn_over(board, color, y, x,  1,-1): return 1
        if self.count_turn_over(board, color, y, x,  1, 1): return 1
        return 0
    
    def legal_list(self,board,color):
        legal = np.empty((0,2), int)
        for i in range(1, 9):
          for j in range(1, 9):
            if self.is_legal(board, color, i, j):
                legal = np.append(legal, np.array([[i,j]]), axis = 0)

        return legal
              
    def exist_legal_move(self, board, color):
        for i in range(1,9):
            for j in range(1,9):
                if self.is_legal(board, color, i, j):return 1
        return 0
    
    def set_turn(self, board, color, y, x):
        count = 0
        for d in range(-1,2):
            for e in range(-1,2):
                if (d == 0 and e == 0):
                    continue
                count = self.count_turn_over(board, color, y, x, d, e)
                for i in range(1, count+1):
                    board[y + i * d][x + i * e] = color
        board[y][x] = color

In [0]:
class Cpu:
    def cpu_randmove(self, board, color):
        l = Play().legal_list(board, color)
        if l.shape[0] == 1:
          y, x = l[0]
        else:
          y, x = l[int(np.random.randint(l.shape[0] - 1,size = 1))]
        return y, x

    def playout(self, board, color):
      player = color
      while True:
        if not Play().exist_legal_move(board, color):
          color = (3 - color)
          if not Play().exist_legal_move(board, color):
            c_1 = np.sum(board == 1)
            c_2 = np.sum(board == 2)
            if c_1 >= c_2:
              color = player
              return np.array([[1,0]])
            else :
              color = player
              return np.array([[0,1]])
        y, x = self.cpu_randmove(board, color)
        Play().set_turn(board, color, y, x)
        color = (3 - color)

    def playout_result(self, board, color, y, x, num):
      tsugi_board = copy.deepcopy(board)
      player = color
      result = np.empty((0, 2), int)
      Play().set_turn(tsugi_board, color, y, x)
      color = (3 - color)
      for i in range(num):
        tsugitsugi_board = copy.deepcopy(tsugi_board)
        result = np.append(result, self.playout(tsugitsugi_board,color), axis = 0)
      if player == 1:
        return np.sum(result[:,0] == 1) / float(num)
      elif player == 2:
        return np.sum(result[:,1] == 1) / float(num)
      
    def decide_move(self, board, color, num):
      rate = []
      l = Play().legal_list(board, color)
      print(l)
      if l.shape[0] == 1:
        y, x = l[0]
      for i in range(l.shape[0]):
        print('{0}手目試行中'.format(i+1))
        q, p = l[i]
        tsugi_board = copy.deepcopy(board)
        rate.append(self.playout_result(tsugi_board, color, q, p, num))
      print(rate)
      y, x = l[rate.index(max(rate))]
      return y, x
      
    

In [0]:
class Othello:
    def game(self):
        ban = Ban().init_ban()
        color = 1
        print('ゲームスタート！')
        while True:
            Ban().draw_ban(ban)
            if not Play().exist_legal_move(ban, color):
                print('打つ手が無いのでパスします')
                color = (3-color)
                if not Play().exist_legal_move(ban, color):
                    print('打つ手が無いのでパスします')
                    break
            y, x = self.get_move(ban, color)
            Play().set_turn(ban, color, y, x)
            color = int(3 - color)
        c_1 = np.sum(ban == 1)
        c_2 = np.sum(ban == 2)
        print('ゲーム終了')
        print('Player 1 %d' % c_1)
        print('Player 2 %d' % c_2)
        if c_1 > c_2:
          print('勝者:Player 1')
        elif c_1<c_2:
          print('勝者:Player 2')
        elif c_1 == c_2:
          print('引き分け')
          
    def cvc_game(self):
        ban = Ban().init_ban()
        color = 1
        while True:
          Ban().draw_ban(ban)
          if not Play().exist_legal_move(ban, color):
            print('打つ手が無いのでパスします')
            color = (3 - color)
            if not Play().exist_legal_move(ban, color):
                print('打つ手が無いのでパスします')
                break
          y, x = Cpu().cpu_randmove(ban, color)
          Play().set_turn(ban, color, y, x)
          color = int(3 - color)
        c_1 = np.sum(ban == 1)
        c_2 = np.sum(ban == 2)
        print('ゲーム終了')
        print('Player 1 %d' % c_1)
        print('Player 2 %d' % c_2)
        if c_1 > c_2:
          print('勝者:Player 1')
        elif c_1 < c_2:
          print('勝者:Player 2')
        elif c_1 == c_2:
          print('引き分け')

    def cvc_mc_game(self, num):  # num=playout try number
      ban = Ban().init_ban()
      color = 1
      for count in range(2):  # 1,2手目はランダム
        Ban().draw_ban(ban)
        y, x = Cpu().cpu_randmove(ban, color)
        Play().set_turn(ban, color, y, x)
        color = 3 - color
      while True:
        Ban().draw_ban(ban)
        if not Play().exist_legal_move(ban, color):
          print('打つ手が無いのでパスします')
          color = (3 - color)
          if not Play().exist_legal_move(ban, color):
            print('打つ手が無いのでパスします')
            break
        y, x = Cpu().decide_move(ban, color, num)
        Play().set_turn(ban, color, y, x)
        color = int(3 - color)
      c_1 = np.sum(ban == 1)
      c_2 = np.sum(ban == 2)
      print('ゲーム終了')
      print('Player 1 %d' % c_1)
      print('Player 2 %d' % c_2)
      if c_1 > c_2:
        print('勝者:Player 1')
      elif c_1 < c_2:
        print('勝者:Player 2')
      elif c_1 == c_2:
        print('引き分け')


          
    def get_move(self, board, color):
        d = {'a':1, 'b':2, 'c':3, 'd':4, 'e':5, 'f':6, 'g':7, 'h':8}
        p = {1 : '●', 2 : '○'}
        if color == 1:
            print('Player {0[1]}(半角数字[半角スペース]半角英字)'.format(p), end = '')
        elif color == 2:
            print('Player {0[2]}(半角数字[半角スペース]半角英字)'.format(p), end='')
        while True:
            print('>>',end = '')
            Y,X = input().split()
            y = int(Y)
            x = d[X]
            if (Play().is_legal(board, color, y, x)):
                return y,x
            else:
                print('illegal hand!', end = '')

In [64]:
if __name__ == '__main__':
  #Othello().game()
  #Othello().cvc_game()
  Othello().cvc_mc_game(500)

  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・ ・ ・ ・ ・ ・ |
2|・ ・ ・ ・ ・ ・ ・ ・ |
3|・ ・ ・ ・ ・ ・ ・ ・ |
4|・ ・ ・ ○ ● ・ ・ ・ |
5|・ ・ ・ ● ○ ・ ・ ・ |
6|・ ・ ・ ・ ・ ・ ・ ・ |
7|・ ・ ・ ・ ・ ・ ・ ・ |
8|・ ・ ・ ・ ・ ・ ・ ・ |
  ------------------------
[[3 4]
 [4 3]
 [5 6]
 [6 5]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
[0.516, 0.498, 0.518, 0.49]
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・ ・ ・ ・ ・ ・ |
2|・ ・ ・ ・ ・ ・ ・ ・ |
3|・ ・ ・ ・ ・ ・ ・ ・ |
4|・ ・ ・ ○ ● ・ ・ ・ |
5|・ ・ ・ ● ● ● ・ ・ |
6|・ ・ ・ ・ ・ ・ ・ ・ |
7|・ ・ ・ ・ ・ ・ ・ ・ |
8|・ ・ ・ ・ ・ ・ ・ ・ |
  ------------------------
[[4 6]
 [6 4]
 [6 6]]
1手目試行中
2手目試行中
3手目試行中
[0.46, 0.516, 0.468]
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・ ・ ・ ・ ・ ・ |
2|・ ・ ・ ・ ・ ・ ・ ・ |
3|・ ・ ・ ・ ・ ・ ・ ・ |
4|・ ・ ・ ○ ● ・ ・ ・ |
5|・ ・ ・ ○ ● ● ・ ・ |
6|・ ・ ・ ○ ・ ・ ・ ・ |
7|・ ・ ・ ・ ・ ・ ・ ・ |
8|・ ・ ・ ・ ・ ・ ・ ・ |
  ------------------------
[[3 3]
 [4 3]
 [5 3]
 [6 3]
 [7 3]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
5手目試行中
[0.478, 0.502, 0.43, 0.456, 0.48]
  a  b  c  d  e  f  g  h 


4手目試行中
5手目試行中
[0.612, 0.608, 0.612, 0.558, 0.622]
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・ ・ ・ ・ ・ ・ |
2|・ ・ ・ ・ ・ ・ ・ ・ |
3|・ ○ ・ ・ ・ ・ ・ ・ |
4|・ ・ ○ ● ● ・ ・ ・ |
5|・ ・ ● ● ● ● ・ ・ |
6|○ ○ ○ ● ・ ・ ・ ・ |
7|・ ・ ・ ○ ・ ・ ・ ・ |
8|・ ・ ・ ・ ○ ・ ・ ・ |
  ------------------------
[[2 1]
 [3 3]
 [4 2]
 [7 1]
 [7 2]
 [7 3]
 [8 4]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
5手目試行中
6手目試行中
7手目試行中
[0.37, 0.424, 0.378, 0.358, 0.45, 0.454, 0.404]
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・ ・ ・ ・ ・ ・ |
2|・ ・ ・ ・ ・ ・ ・ ・ |
3|・ ○ ・ ・ ・ ・ ・ ・ |
4|・ ・ ○ ● ● ・ ・ ・ |
5|・ ・ ● ● ● ● ・ ・ |
6|○ ○ ● ● ・ ・ ・ ・ |
7|・ ・ ● ○ ・ ・ ・ ・ |
8|・ ・ ・ ・ ○ ・ ・ ・ |
  ------------------------
[[3 4]
 [3 5]
 [4 6]
 [5 2]
 [6 5]
 [7 2]
 [8 3]
 [8 4]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
5手目試行中
6手目試行中
7手目試行中
8手目試行中
[0.534, 0.552, 0.54, 0.55, 0.544, 0.504, 0.6, 0.582]
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・ ・ ・ ・ ・ ・ |
2|・ ・ ・ ・ ・ ・ ・ ・ |
3|・ ○ ・ ・ ・ ・ ・ ・ |
4|・ ・ ○ ● ● ・ ・ ・ |
5|・ ・ ○ ● ● ● ・ ・ |
6|○ 

3手目試行中
4手目試行中
5手目試行中
6手目試行中
7手目試行中
8手目試行中
9手目試行中
10手目試行中
11手目試行中
[0.444, 0.518, 0.512, 0.522, 0.566, 0.504, 0.54, 0.532, 0.548, 0.442, 0.64]
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・ ・ ・ ・ ・ ・ |
2|・ ● ・ ・ ・ ・ ・ ・ |
3|・ ● ・ ・ ・ ・ ・ ・ |
4|・ ● ○ ● ● ・ ・ ・ |
5|・ ● ● ● ● ● ・ ・ |
6|○ ● ○ ○ ○ ・ ・ ・ |
7|・ ○ ○ ○ ・ ・ ・ ・ |
8|○ ・ ○ ・ ○ ・ ・ ・ |
  ------------------------
[[3 3]
 [3 4]
 [6 6]
 [7 5]
 [7 6]
 [8 2]
 [8 4]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
5手目試行中
6手目試行中
7手目試行中
[0.336, 0.356, 0.376, 0.348, 0.366, 0.366, 0.424]
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・ ・ ・ ・ ・ ・ |
2|・ ● ・ ・ ・ ・ ・ ・ |
3|・ ● ・ ・ ・ ・ ・ ・ |
4|・ ● ○ ● ● ・ ・ ・ |
5|・ ● ● ● ● ● ・ ・ |
6|○ ● ○ ● ○ ・ ・ ・ |
7|・ ○ ● ● ・ ・ ・ ・ |
8|○ ・ ○ ● ○ ・ ・ ・ |
  ------------------------
[[1 2]
 [2 1]
 [3 5]
 [3 6]
 [4 1]
 [4 6]
 [4 7]
 [7 5]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
5手目試行中
6手目試行中
7手目試行中
8手目試行中
[0.636, 0.536, 0.654, 0.634, 0.616, 0.592, 0.622, 0.616]
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・

4手目試行中
5手目試行中
6手目試行中
7手目試行中
8手目試行中
9手目試行中
10手目試行中
11手目試行中
12手目試行中
[0.232, 0.312, 0.356, 0.316, 0.288, 0.314, 0.27, 0.222, 0.312, 0.314, 0.324, 0.256]
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ○ ・ ・ ・ ・ ・ ・ |
2|・ ○ ・ ・ ・ ● ・ ・ |
3|・ ○ ● ・ ● ・ ・ ・ |
4|・ ○ ● ● ○ ・ ・ ・ |
5|・ ○ ● ● ○ ● ・ ・ |
6|○ ○ ○ ○ ○ ・ ・ ・ |
7|● ○ ○ ● ・ ・ ・ ・ |
8|○ ○ ○ ● ○ ・ ・ ・ |
  ------------------------
[[1 7]
 [2 3]
 [2 4]
 [2 5]
 [3 4]
 [4 7]
 [5 7]
 [6 7]
 [7 5]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
5手目試行中
6手目試行中
7手目試行中
8手目試行中
9手目試行中
[0.656, 0.616, 0.65, 0.652, 0.654, 0.632, 0.606, 0.674, 0.65]
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ○ ・ ・ ・ ・ ・ ・ |
2|・ ○ ・ ・ ・ ● ・ ・ |
3|・ ○ ● ・ ● ・ ・ ・ |
4|・ ○ ● ● ○ ・ ・ ・ |
5|・ ○ ● ● ○ ○ ・ ・ |
6|○ ○ ○ ○ ○ ・ ○ ・ |
7|● ○ ○ ● ・ ・ ・ ・ |
8|○ ○ ○ ● ○ ・ ・ ・ |
  ------------------------
[[1 1]
 [2 1]
 [3 1]
 [3 6]
 [4 1]
 [4 6]
 [4 7]
 [5 1]
 [5 7]
 [6 6]
 [7 5]
 [7 6]
 [8 6]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
5手目試行中
6手目試行中
7手目試行中
8手目試行中
9手目試行中
10手目試行中
11手目試行中
12手目試行中
13

[0.464, 0.492, 0.452, 0.44]
  a  b  c  d  e  f  g  h 
  ------------------------
1|● ● ● ・ ・ ・ ○ ・ |
2|● ● ・ ○ ・ ○ ・ ・ |
3|・ ● ○ ○ ○ ・ ・ ・ |
4|・ ○ ○ ○ ○ ・ ・ ・ |
5|・ ○ ○ ● ○ ○ ・ ・ |
6|○ ○ ○ ○ ○ ・ ○ ・ |
7|● ○ ○ ● ・ ・ ・ ・ |
8|○ ○ ○ ● ○ ・ ・ ・ |
  ------------------------
[[1 4]
 [3 6]
 [4 1]
 [4 6]
 [4 7]
 [5 1]
 [5 7]
 [6 6]
 [7 6]
 [8 6]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
5手目試行中
6手目試行中
7手目試行中
8手目試行中
9手目試行中
10手目試行中
[0.592, 0.558, 0.438, 0.56, 0.586, 0.446, 0.534, 0.6, 0.596, 0.56]
  a  b  c  d  e  f  g  h 
  ------------------------
1|● ● ● ・ ・ ・ ○ ・ |
2|● ● ・ ○ ・ ○ ・ ・ |
3|・ ● ● ○ ○ ・ ・ ・ |
4|・ ○ ○ ● ○ ・ ・ ・ |
5|・ ○ ○ ● ● ○ ・ ・ |
6|○ ○ ○ ○ ○ ● ○ ・ |
7|● ○ ○ ● ・ ・ ・ ・ |
8|○ ○ ○ ● ○ ・ ・ ・ |
  ------------------------
[[2 3]
 [3 1]
 [4 6]
 [7 5]
 [7 6]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
5手目試行中
[0.42, 0.372, 0.43, 0.388, 0.454]
  a  b  c  d  e  f  g  h 
  ------------------------
1|● ● ● ・ ・ ・ ○ ・ |
2|● ● ・ ○ ・ ○ ・ ・ |
3|・ ● ● ○ ○ ・ ・ ・ |
4|・ ○ ○ ● ○ ・ ・ ・ |
5|・ ○ ○ ● ● ○ ・ ・ |
6|○ ○ ○ ○ ○ ○ ○ ・ |
7|● ○

11手目試行中
12手目試行中
[0.58, 0.574, 0.524, 0.634, 0.514, 0.576, 0.47, 0.564, 0.532, 0.57, 0.516, 0.526]
  a  b  c  d  e  f  g  h 
  ------------------------
1|● ● ● ● ・ ・ ○ ・ |
2|● ● ・ ● ○ ○ ・ ・ |
3|● ○ ○ ○ ○ ・ ・ ・ |
4|● ● ● ● ○ ・ ・ ・ |
5|・ ● ○ ● ● ○ ・ ・ |
6|○ ○ ● ○ ● ○ ○ ・ |
7|● ○ ○ ● ● ○ ・ ・ |
8|○ ○ ○ ● ○ ・ ・ ・ |
  ------------------------
[[1 5]
 [2 3]
 [4 6]
 [5 1]
 [8 6]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
5手目試行中
[0.334, 0.37, 0.342, 0.48, 0.286]
  a  b  c  d  e  f  g  h 
  ------------------------
1|● ● ● ● ・ ・ ○ ・ |
2|● ● ・ ● ○ ○ ・ ・ |
3|● ○ ○ ○ ○ ・ ・ ・ |
4|● ○ ● ● ○ ・ ・ ・ |
5|○ ○ ○ ● ● ○ ・ ・ |
6|○ ○ ● ○ ● ○ ○ ・ |
7|● ○ ○ ● ● ○ ・ ・ |
8|○ ○ ○ ● ○ ・ ・ ・ |
  ------------------------
[[1 5]
 [1 6]
 [2 3]
 [2 7]
 [3 6]
 [4 6]
 [4 7]
 [5 7]
 [6 8]
 [7 7]
 [8 6]
 [8 7]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
5手目試行中
6手目試行中
7手目試行中
8手目試行中
9手目試行中
10手目試行中
11手目試行中
12手目試行中
[0.538, 0.596, 0.57, 0.54, 0.488, 0.446, 0.51, 0.472, 0.476, 0.538, 0.438, 0.508]
  a  b  c  d  e  f  g  h 
  ------------------------
1|● ● ● ● 

6手目試行中
7手目試行中
8手目試行中
9手目試行中
10手目試行中
11手目試行中
[0.806, 0.626, 0.68, 0.648, 0.684, 0.634, 0.616, 0.564, 0.588, 0.666, 0.574]
  a  b  c  d  e  f  g  h 
  ------------------------
1|● ● ● ● ● ● ● ● |
2|● ● ○ ● ● ○ ○ ・ |
3|● ● ● ● ● ○ ・ ・ |
4|● ○ ● ● ○ ・ ・ ・ |
5|○ ○ ○ ○ ● ○ ・ ・ |
6|○ ○ ○ ○ ● ○ ○ ・ |
7|● ○ ○ ● ○ ○ ・ ・ |
8|○ ○ ○ ● ○ ○ ・ ・ |
  ------------------------
[[4 6]]
1手目試行中
[0.242]
  a  b  c  d  e  f  g  h 
  ------------------------
1|● ● ● ● ● ● ● ● |
2|● ● ○ ● ● ○ ○ ・ |
3|● ● ● ● ● ○ ・ ・ |
4|● ○ ● ● ○ ○ ・ ・ |
5|○ ○ ○ ○ ○ ○ ・ ・ |
6|○ ○ ○ ○ ● ○ ○ ・ |
7|● ○ ○ ● ○ ○ ・ ・ |
8|○ ○ ○ ● ○ ○ ・ ・ |
  ------------------------
[[2 8]
 [3 7]
 [3 8]
 [4 7]
 [5 7]
 [6 8]
 [7 7]
 [7 8]
 [8 7]]
1手目試行中
2手目試行中
3手目試行中
4手目試行中
5手目試行中
6手目試行中
7手目試行中
8手目試行中
9手目試行中
[0.934, 0.694, 0.792, 0.81, 0.742, 0.728, 0.658, 0.778, 0.73]
  a  b  c  d  e  f  g  h 
  ------------------------
1|● ● ● ● ● ● ● ● |
2|● ● ○ ● ● ● ● ● |
3|● ● ● ● ● ○ ・ ・ |
4|● ○ ● ● ○ ○ ・ ・ |
5|○ ○ ○ ○ ○ ○ ・ ・ |
6|○ ○ ○ ○ ● ○ ○ ・ |
7|● ○ ○ ● ○ ○ 

[1.0, 1.0, 1.0]
  a  b  c  d  e  f  g  h 
  ------------------------
1|● ● ● ● ● ● ● ● |
2|● ● ○ ● ● ● ● ● |
3|● ● ● ● ● ● ● ● |
4|● ○ ● ● ● ● ● ● |
5|○ ○ ○ ○ ○ ● ○ ● |
6|○ ○ ○ ○ ● ● ● ● |
7|● ○ ○ ● ● ● ● ● |
8|○ ○ ○ ● ○ ○ ・ ・ |
  ------------------------
[[8 7]
 [8 8]]
1手目試行中
2手目試行中
[0.0, 0.0]
  a  b  c  d  e  f  g  h 
  ------------------------
1|● ● ● ● ● ● ● ● |
2|● ● ○ ● ● ● ● ● |
3|● ● ● ● ● ● ● ● |
4|● ○ ● ● ● ● ● ● |
5|○ ○ ○ ○ ○ ● ○ ● |
6|○ ○ ○ ○ ○ ● ○ ● |
7|● ○ ○ ● ● ○ ○ ● |
8|○ ○ ○ ● ○ ○ ○ ・ |
  ------------------------
[[8 8]]
1手目試行中
[1.0]
  a  b  c  d  e  f  g  h 
  ------------------------
1|● ● ● ● ● ● ● ● |
2|● ● ○ ● ● ● ● ● |
3|● ● ● ● ● ● ● ● |
4|● ○ ● ● ● ● ● ● |
5|○ ○ ○ ○ ○ ● ○ ● |
6|○ ○ ○ ○ ○ ● ○ ● |
7|● ○ ○ ● ● ○ ● ● |
8|○ ○ ○ ● ● ● ● ● |
  ------------------------
打つ手が無いのでパスします
打つ手が無いのでパスします
ゲーム終了
Player 1 44
Player 2 20
勝者:Player 1
